In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
from bs4 import BeautifulSoup

In [ ]:
def login(browser):
    username = input('Enter Username: ')
    password = input('Enter Password: ')
    browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
    time.sleep(3)
    browser.find_element_by_name('session_key').send_keys(username + Keys.RETURN)
    browser.find_element_by_name('session_password').send_keys(password + Keys.RETURN)
    time.sleep(3)

In [ ]:
def connections_scraper(browser):
    connections_page = "https://www.linkedin.com/search/results/people/?facetNetwork=%5B%22F%22%5D&origin=MEMBER_PROFILE_CANNED_SEARCH"
    browser.get(connections_page)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    conn_num = soup.find_all('h3', class_='search-results__total')
    num = int(conn_num[0].text.strip().split()[0])
    time.sleep(3)
    i = 2
    x = 1
    names = []
    titles = []
    locations = []
    profiles = []
    print('\nScraping your connections...\n')
    while True:
        browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(.75)
        browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(.75)
        browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(.75)
        browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(.75)
        browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        name_tag = soup.find_all('span', class_='name actor-name')
        title_tag = soup.find_all('p', class_='subline-level-1')
        location_tag = soup.find_all('p', class_= 'subline-level-2')
        profile_tag = soup.find_all('a', class_= 'search-result__result-link')
        names += list(map(lambda x: x.text, name_tag))
        titles += list(map(lambda x: x.text.replace('\n','').strip(), title_tag))
        locations += list(map(lambda x: x.text.replace('\n','').strip(), location_tag))
        profiles += list(map(lambda x: 'https://linkedin.com' + x['href'], profile_tag))[::2]
        if len(names)>=num:
            break
        y = x
        x = len(names)
        if x==y:
            break
        browser.get('https://www.linkedin.com/search/results/people/?facetNetwork=%5B%22F%22%5D&origin=MEMBER_PROFILE_CANNED_SEARCH&page='+str(i))
        i+=1
        time.sleep(3)
    df = pd.DataFrame({'Name':names, 'Title':titles, 'Location':locations, 'Profile':profiles})
    return df

In [ ]:
browser = webdriver.Chrome(executable_path="E:/RnD/linkedin_scraper/chromedriver", options= webdriver.ChromeOptions())
login(browser)
connections = connections_scraper(browser)
browser.quit()
connections.to_csv('connections.csv')

In [ ]:
def profile_scraper(df, browser):
    num_projects = []
    num_languages = []
    top_skills = []
    num_connections = []
    positions = []
    company = []
    duration = []
    institutes = []
    courses = []
    year_range = []
    ex_profiles = []
    ed_profiles = []
    for profile in df['Profile']:
        try:
            browser.get(profile)
            time.sleep(2)
            browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(.75)
            browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(.75)
            browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(.75)
            browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(.75)
            browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(.75)
            browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(.75)
            browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(.75)
            browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(.75)
            soup = BeautifulSoup(browser.page_source, 'lxml')

            conn_tag = soup.find_all('span', class_='t-16 t-bold')
            if conn_tag[0].text.strip().split()[0].isdigit() or conn_tag[0].text.strip().split()[0] == '500+':
                num_connections.append(conn_tag[0].text.strip().split()[0])
            elif len(soup.find_all('span', class_='t-16 t-black t-normal'))>0:
                num_connections.append(soup.find_all('span', class_='t-16 t-black t-normal')[0].text.strip().split()[0])
            else:
                num_connections.append(None)

            accom_tag = soup.find_all('h3', class_='pv-accomplishments-block__count t-32 t-black t-normal pr3')
            np = 0
            nl = 2
            for at in accom_tag:
                if at.text.strip().split('\n')[0].split()[-1] == 'projects' or at.text.strip().split('\n')[0].split()[-1] == 'project':
                    np = int(at.text.strip().split('\n')[1])
                if at.text.strip().split('\n')[0].split()[-1] == 'languages' or at.text.strip().split('\n')[0].split()[-1] == 'language':
                    nl = int(at.text.strip().split('\n')[1])
            num_projects.append(np)
            num_languages.append(nl)

            skills_tag = soup.find_all('span', class_='pv-skill-category-entity__name-text')
            ts = []
            for st in skills_tag:
                ts.append(st.text.strip())
            top_skills.append(ts)

            position_tag = soup.find_all('h3', class_='t-16 t-black t-bold')
            ex_pos = list(map(lambda x: x.text.strip(), position_tag))
            company_tag = soup.find_all('p', class_='pv-entity__secondary-title t-14 t-black t-normal')
            ex_comp = list(map(lambda x: x.text.strip().split('\n')[0], company_tag))
            ex_duration_tag = soup.find_all('span', class_='pv-entity__bullet-item-v2')
            durr = []
            for dur in ex_duration_tag:
                d_list = dur.text.strip().split()
                if d_list[0].isdigit():
                    if len(d_list)==2:
                        if d_list[1] == 'mo' or d_list[1]=='mos':
                            durr.append(int(d_list[0]))
                        if d_list[1] == 'yr' or d_list[1]=='yrs':
                            durr.append(int(d_list[0])*12)
                    if len(d_list)==4:
                        durr.append((int(d_list[0])*12)+int(d_list[2]))
                else:
                    durr.append(None)
            x = min(len(ex_comp), len(ex_pos), len(durr))
            ex_comp = ex_comp[:x]
            ex_pos = ex_pos[:x]
            durr = durr[:x]
            ex_profiles += [profile]*x
            positions += ex_pos
            company += ex_comp
            duration += durr

            institute_tag = soup.find_all('h3', class_='pv-entity__school-name t-16 t-black t-bold')
            inst = list(map(lambda x: x.text.strip(), institute_tag))
            course_tag = soup.find_all('p', class_='pv-entity__secondary-title pv-entity__degree-name t-14 t-black t-normal')
            course_t = list(map(lambda x: x.text.strip().split('\n')[1], course_tag))
            ed_date_tag = soup.find_all('p', class_='pv-entity__dates t-14 t-black--light t-normal')
            ed_dates = list(map(lambda x: x.text.strip().split('\n')[-1], ed_date_tag))
            y = min(len(inst), len(course_t), len(ed_dates))
            inst = inst[:y]
            course_t = course_t[:y]
            ed_dates = ed_dates[:y]
            ed_profiles += [profile]*y
            institutes += inst
            courses += course_t
            year_range += ed_dates
        except:
            continue
    df['Number of connections'] = num_connections
    df['Number of Projects'] = num_projects
    df['Number of Languages known'] = num_languages
    df['Top Skills'] = top_skills
    exp_df = pd.DataFrame({'Profile':ex_profiles, 'Position':positions, 'Company':company, 'Duration':duration})
    ed_df = pd.DataFrame({'Profile':ed_profiles, 'Institute':institutes, 'Degree':courses, 'Year range':year_range})
    
    return df, exp_df, ed_df

In [ ]:
browser = webdriver.Chrome(executable_path="E:/RnD/linkedin_scraper/chromedriver", options= webdriver.ChromeOptions())
login(browser)
data = pd.read_csv('connections.csv')
df, exp, ed = profile_scraper(data, browser)
browser.quit()
df.to_csv('connections_data.csv', index=False)
exp.to_csv('experience.csv', index=False)
ed.to_csv('education.csv', index=False)

In [ ]:
browser = webdriver.Chrome(executable_path="E:/RnD/linkedin_scraper/chromedriver", options= webdriver.ChromeOptions())
login(browser)
connections = connections_scraper(browser)
conn, exp, ed = profile_scraper(connections, browser)
browser.quit()